# Анализ базы данных сервиса для чтения книг по подписке

**Цели исследования:** Проанализировать базу данных для дальнейшего формирования ценностного предложения продукта.

**План:**
1. Загрузка БД
2. Просмотр общего содержания таблиц
3. Исследование данных
    * Количество книг с 2000 года
    * Обзоры и средняя оценка книг
    * Крупные издательства
    * Авторы с высоким рейтингом
    * Количество ревьюеров-экспертов
4. Общий вывод

### Таблицы

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
con=engine.connect()

In [2]:
# чтобы выполнить SQL-запрос, используем Pandas
def sql(query):
    return pd.io.sql.read_sql(sql=text(query), con = con)

In [3]:
#books
sql('''SELECT count(book_id) FROM books''')

,count
0,1000


In [4]:
#books
sql('''SELECT * FROM books LIMIT 5''')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
#authors
sql('''SELECT count(author_id) FROM authors''')

,count
0,636


In [6]:
#authors
sql('''SELECT * FROM authors LIMIT 5''')

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [7]:
#ratings
sql('''SELECT count(rating_id) FROM ratings''')

,count
0,6456


In [8]:
#ratings
sql('''SELECT * FROM ratings LIMIT 5''')

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [9]:
#reviews
sql('''SELECT count(review_id) FROM reviews''')

,count
0,2793


In [10]:
#reviews
sql('''SELECT * FROM reviews LIMIT 5''')

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [11]:
#publishers
sql('''SELECT count(publisher_id) FROM publishers''')

,count
0,340


In [12]:
#publishers
sql('''SELECT * FROM publishers LIMIT 5''')

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### Исследование (+выводы)

1. Посчитайте, сколько книг вышло после 1 января 2000 года;

In [13]:
sql('''
SELECT COUNT(book_id) AS book_count
FROM books 
WHERE publication_date > '2000-01-01'
''')

,book_count
0,819


**Выводы 1:** Из 1000 книг в базе данных 819 вышли после 1 января 2000 года.

2. Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [14]:
sql('''
SELECT b.title,
b.book_id,
COUNT(DISTINCT(rev.review_id)) AS review_count,
ROUND(AVG(rat.rating), 1) AS rating
FROM books b LEFT JOIN ratings rat ON b.book_id=rat.book_id 
LEFT JOIN reviews rev ON rev.book_id=rat.book_id
GROUP BY b.book_id, b.title
ORDER BY review_count DESC
''')

,title,book_id,review_count,rating
0,Twilight (Twilight #1),948,7,3.7
1,Water for Elephants,963,6,4.0
2,The Glass Castle,734,6,4.2
3,Harry Potter and the Prisoner of Azkaban (Harr...,302,6,4.4
4,The Curious Incident of the Dog in the Night-Time,695,6,4.1
...,...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,83,0,3.7
996,The Natural Way to Draw,808,0,3.0
997,The Cat in the Hat and Other Dr. Seuss Favorites,672,0,5.0
998,Essential Tales and Poems,221,0,4.0


**Выводы 2:** В базе данных есть хотя бы 1 оценка по каждой книге, обзоры есть на 99% книг (994).\
Три самых популярных с точки зрения кол-ва отзывов - Twilight, Water for Elephants, The Glass Castle.

3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [15]:
sql('''
SELECT p.publisher,
COUNT(b.book_id) as book_count
FROM publishers p JOIN books b ON p.publisher_id=b.publisher_id
WHERE b.num_pages>50
GROUP BY p.publisher
ORDER BY book_count DESC
''')

,publisher,book_count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
...,...,...
329,Turtleback,1
330,Atheneum Books for Young Readers: Richard Jack...,1
331,Penguin Signet,1
332,Victor Gollancz,1


**Выводы 3:** В базе данных содержится информация о 334 издательствах. Количество изданных книг (более 50 страниц) от 1 до 42 штук.\
Самое производительное издательство - Penguin Books (42 книги).

4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [16]:
sql('''
SELECT author,
ROUND(AVG(avg_rating),2) as rating
FROM (SELECT a.author,
b.book_id,
AVG(r.rating) as avg_rating,
COUNT(r.rating_id) AS quantity
FROM authors a 
JOIN books b ON a.author_id=b.author_id
JOIN ratings r ON r.book_id=b.book_id
GROUP BY a.author, b.book_id) b
WHERE b.quantity >= 50
GROUP BY author
ORDER BY rating DESC
''')

,author,rating
0,J.K. Rowling/Mary GrandPré,4.28
1,Markus Zusak/Cao Xuân Việt Khương,4.26
2,J.R.R. Tolkien,4.26
3,Louisa May Alcott,4.19
4,Rick Riordan,4.08
5,William Golding,3.90
6,J.D. Salinger,3.83
7,William Shakespeare/Paul Werstine/Barbara A. M...,3.79
8,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.79
9,Lois Lowry,3.75


**Выводы 4:** В БД 13 авторов, издавших более 50 книг (включительно).\
Средний рейтинг по книгам варьируется от 3,6 до 4,2.\
Самая высокая оценка у автора J.K. Rowling/Mary GrandPré. Низкая - John Steinbeck.

5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [17]:
sql('''
SELECT AVG(review_count) AS average_review_count
FROM(SELECT c.username,
COUNT(rev.review_id) as review_count
FROM (SELECT rat.username,
COUNT(rat.rating_id) AS rating_quantity
FROM ratings rat
GROUP BY rat.username) c JOIN reviews rev ON c.username=rev.username
WHERE rating_quantity > 48
GROUP BY c.username) d
''')

,average_review_count
0,24.0


**Выводы 5:** Среднее количество обзоров от пользователей-экспертов (поставивших больше 48 оценок книгам) составляет 24 штуки.

### Общий вывод

БД содержит 5 таблиц: books,authors, ratings, reviews, publishers.
- Таблица books содержит информацию о 1000 книг, 819 из которых были выпущены после 1 января 2000 года.
- Для большинства книг (99%) есть хотя бы 1 текстовый обзор (в таблице reviews, суммарно отзывов в таблице 2793), для всех книг есть хотя бы 1 оценка(таблица ratings, суммарно поставлено 6456 оценок). Три самых популярных книги с точки зрения кол-ва отзывов - Twilight, Water for Elephants, The Glass Castle
- Таблица publishers включает информацию о 334 издательствах. Количество изданных книг (более 50 страниц) от 1 до 42 штук. Самое производительное издательство - Penguin Books (42 книги).
- В таблице authors 13 крупных авторов, издавших более 50 книг (включительно). Средний рейтинг по книгам варьируется от 3,6 до 4,2. Самая высокая оценка у автора J.K. Rowling/Mary GrandPré. Низкая - John Steinbeck.
- Пользователи-эксперты (поставившие больше 48 оценок книгам) в среднем написали по 24 обзора книгам.